In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy = 1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import TripletNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)
triplet_network = TripletNetwork(metric_network)  #metric learning   
triplet_network.cuda(GPU)
attribute_network.cuda(GPU)
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
attribute_network_optim=torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
attribute_network_scheduler = StepLR(attribute_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float()) # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext).view(-1, class_num)
        #relations = F.cos(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
        #rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        #total_rewards += np.sum(rewards)
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    
    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    #print(np.count_nonzero(per_class_num))
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)
    attribute_network_scheduler.step(episode)
    

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    batch_attributes_negative = torch.empty_like(batch_attributes)
    
    
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=batch_labels.numpy()[i])[0], replace=True)
        batch_features_negative[i] = batch_features[pick]
    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    
    
    for i in range(batch_attributes_negative.size()[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=batch_labels.numpy()[i])[0], replace=True)
        batch_attributes_negative[i] = batch_attributes[pick]
    
    
    #print(batch_attributes.size())
    #print(batch_attributes_negative.size())
    
    
    #batch_attributes_negative = torch.from_numpy(batch_features_negative)
    batch_attributes_negative = Variable(batch_attributes_negative).cuda(GPU).float()  # 32*2048
    
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    #print((batch_attributes_negative).size())
    #print(attribute_network(batch_attributes_negative).size())
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(attribute_network(batch_attributes), batch_features, batch_features_negative)
    dista2, distb2, embedded_x2, embedded_y2, embedded_z2 = triplet_network(batch_features,
                                                                            attribute_network(batch_attributes), 
                                                                            attribute_network(batch_attributes_negative))
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target) + criterion(dista2, distb2, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet #+ Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    attribute_network.zero_grad()
    metric_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    attribute_network_optim.step()
    
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss)
        writer.add_scalar('data/loss', loss, episode)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy_per_class(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)
        writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
writer.close()


training...
episode: 1000 loss tensor(0.2492, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.11115367231638418
gzsl: unseen=0.1082 , seen=0.0186 , h=0.0317
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.1425, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.1436327683615819
gzsl: unseen=0.1356 , seen=0.0564 , h=0.0796
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.0100, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.16296079897371427
gzsl: unseen=0.1442 , seen=0.1285 , h=0.1359
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.0694, device='cuda:0', grad_fn=<AddBackward0>)
Testing...


class num: 200
class num: 200
zsl: 0.2807292700947256
gzsl: unseen=0.2056 , seen=0.3368 , h=0.2554
____________________________________________________________________________________________________
episode: 29000 loss tensor(0.0330, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.25775507500487044
gzsl: unseen=0.1925 , seen=0.3318 , h=0.2436
____________________________________________________________________________________________________
episode: 30000 loss tensor(0.0872, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.287374616337498
gzsl: unseen=0.2088 , seen=0.3779 , h=0.2690
____________________________________________________________________________________________________
episode: 31000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.30366661520534904
gzsl: unseen=0.2222 , seen=0.3655 , h=0.2763
_

episode: 56000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3349592793945208
gzsl: unseen=0.2622 , seen=0.3608 , h=0.3037
____________________________________________________________________________________________________
episode: 57000 loss tensor(0.0219, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.31853246731287377
gzsl: unseen=0.2601 , seen=0.3436 , h=0.2961
____________________________________________________________________________________________________
episode: 58000 loss tensor(0.0039, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3364461181625369
gzsl: unseen=0.2606 , seen=0.3500 , h=0.2987
____________________________________________________________________________________________________
episode: 59000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
cla

class num: 200
zsl: 0.3567426917839906
gzsl: unseen=0.2865 , seen=0.3332 , h=0.3081
____________________________________________________________________________________________________
episode: 84000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.36576300650670596
gzsl: unseen=0.3121 , seen=0.3380 , h=0.3246
____________________________________________________________________________________________________
episode: 85000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3578971383831589
gzsl: unseen=0.2977 , seen=0.3517 , h=0.3225
____________________________________________________________________________________________________
episode: 86000 loss tensor(0.0061, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.36015480399276817
gzsl: unseen=0.2925 , seen=0.3449 , h=0.3165
___________________

episode: 111000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.39962944884403706
gzsl: unseen=0.3388 , seen=0.3565 , h=0.3474
____________________________________________________________________________________________________
episode: 112000 loss tensor(0.0005, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.39446228767611724
gzsl: unseen=0.3307 , seen=0.3546 , h=0.3422
____________________________________________________________________________________________________
episode: 113000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.38640755767212104
gzsl: unseen=0.3317 , seen=0.3559 , h=0.3434
____________________________________________________________________________________________________
episode: 114000 loss tensor(0.0010, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 

...

episode: 387000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3851078748418746
gzsl: unseen=0.3516 , seen=0.3310 , h=0.3410
____________________________________________________________________________________________________
episode: 388000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.38320707052248465
gzsl: unseen=0.3549 , seen=0.3234 , h=0.3384
____________________________________________________________________________________________________
episode: 389000 loss tensor(0.0002, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.37679172255208226
gzsl: unseen=0.3456 , seen=0.3232 , h=0.3341
____________________________________________________________________________________________________
episode: 390000 loss tensor(0.0002, device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 5

zsl: 0.3797905346883413
gzsl: unseen=0.3525 , seen=0.3009 , h=0.3247
____________________________________________________________________________________________________
episode: 415000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.38083876124206617
gzsl: unseen=0.3559 , seen=0.3006 , h=0.3259
____________________________________________________________________________________________________
episode: 416000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3722169221616064
gzsl: unseen=0.3423 , seen=0.3036 , h=0.3218
____________________________________________________________________________________________________
episode: 417000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.36803370501007426
gzsl: unseen=0.3464 , seen=0.2982 , h=0.3205
___________________________________

episode: 442000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.36714415023974145
gzsl: unseen=0.3392 , seen=0.3017 , h=0.3193
____________________________________________________________________________________________________
episode: 443000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3742389374708233
gzsl: unseen=0.3473 , seen=0.3157 , h=0.3308
____________________________________________________________________________________________________
episode: 444000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3779332877533092
gzsl: unseen=0.3504 , seen=0.3152 , h=0.3319
____________________________________________________________________________________________________
episode: 445000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class n

episode: 470000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3688306235273793
gzsl: unseen=0.3436 , seen=0.3003 , h=0.3205
____________________________________________________________________________________________________
episode: 471000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.37150199433610437
gzsl: unseen=0.3513 , seen=0.3009 , h=0.3241
____________________________________________________________________________________________________
episode: 472000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3709097425201238
gzsl: unseen=0.3453 , seen=0.3006 , h=0.3214
____________________________________________________________________________________________________
episode: 473000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class n

episode: 498000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.3788264366240746
gzsl: unseen=0.3488 , seen=0.3091 , h=0.3278
____________________________________________________________________________________________________
episode: 499000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.36947946325845704
gzsl: unseen=0.3432 , seen=0.3086 , h=0.3250
____________________________________________________________________________________________________
episode: 500000 loss tensor(0., device='cuda:0', grad_fn=<AddBackward0>)
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.37394838403585906
gzsl: unseen=0.3436 , seen=0.3161 , h=0.3293
____________________________________________________________________________________________________
